# DNA Sequence Alignment Using Dynamic Programming
In this project, I used the [Needleman-Wunsch algorithm](https://en.wikipedia.org/wiki/Needleman%E2%80%93Wunsch_algorithm) to find the optimal global pairwise alignment between two DNA sequences. The algorithm uses a nucleotide match score, nucleotide mismatch penalty, and gap penalty to score possible alignments. <br> <br>
Through dynamic programming, I filled an alignment scoring matrix by building upon the optimal alignments of smaller portions of the two DNA sequences. By recursively tracing back through the optimal path(s) in the scoring matrix, I obtained the optimal sequence alignment(s).

![matrix figure](image1.png) ![matrix results](image2.png)

To find all optimal alignments after scoring, I stored the directional movements at each cell in a separate matrix. 7 coded values were used to represent the 7 possibilities: <br>

Coding values for the trace-back matrix: <br>
1 - came from left neighbor <br>
2 - came from top-left neighbor <br>
3 - came from top neighbor <br>
---- ties (coming from multiple neighbors) ---- <br>
4 - came from left and top-left neighbors <br>
5 - came from top-left and top neighbors <br>
6 - came from left and top neighbors <br>
7 - came from all three neighbors <br>

In [1]:
import numpy as np

**Helper Function 1**

In [2]:
#takes input of sequences X and Y, nucleotide match score, nucleotide mismatch penalty, and gap penalty
#outputs scoring matrix and trace-back matrix with direction history

def fill_matrices(X: list, Y: list, s_match: int, p_mismatch: int, p_gap: int):
    matrix = np.zeros((len(X)+1, len(Y)+1), dtype = int) #matrix for scores
    matrix_trace = np.zeros((len(X)+1, len(Y)+1), dtype = int) #matrix to trace back for optimal alignment
    
    #initialize both matrices
    for i in range(0, len(X)+1): #fill the first column
        matrix[i][0] = p_gap*i
        matrix_trace[i][0] = 3
    for j in range(0, len(Y)+1): #fill the first row
        matrix[0][j] = p_gap*j
        matrix_trace[0][j] = 1
    
    #use dynamic programming to fill the scoring matrix
    for y in range(0, len(Y)):
        for x in range(0, len(X)):
            #obtain score when nucleotides are aligned (coming diagonally from the top-left)
            diag = 0
            if(X[x] == Y[y]):
                diag = matrix[x][y] + s_match
            else:
                diag = matrix[x][y] + p_mismatch
            
            #obtain scores when gaps are placed (coming from the left cell or top cell)
            left = matrix[x+1][y] + p_gap
            top = matrix[x][y+1] + p_gap
            
            #obtain the optimal local score and fill value into matrix
            max_score = max([diag, left, top])
            matrix[x+1][y+1] = max_score

            #storing direction history (see key for coded values above)
            if(diag == max_score and left != max_score and top != max_score):
                matrix_trace[x+1][y+1] = 2
            elif(diag != max_score and left == max_score and top != max_score):
                matrix_trace[x+1][y+1] = 1
            elif(diag != max_score and left != max_score and top == max_score):
                matrix_trace[x+1][y+1] = 3
            elif(diag == max_score and left == max_score and top != max_score):
                matrix_trace[x+1][y+1] = 4
            elif(diag == max_score and left != max_score and top == max_score):
                matrix_trace[x+1][y+1] = 5
            elif(diag != max_score and left == max_score and top == max_score):
                matrix_trace[x+1][y+1] = 6
            else: 
                matrix_trace[x+1][y+1] = 7

    return matrix, matrix_trace

**Helper Function 2**

In [3]:
#recursive helper function to trace back all optimal paths and store the corresponding alignments 
def recursive_paths(x_pos: int, y_pos: int, curr_strX: str, curr_strY: str, 
                    alignments: list, X: list, Y: list, matrix_trace: np.ndarray) -> None:
    
    if(x_pos == 0 and y_pos == 0): #base case: once the beginning is reached, store optimal alignment
        alignments.append([curr_strY, curr_strX]) 
        return
    
    if(matrix_trace[x_pos][y_pos] == 1): #came from left neighbor
        recursive_paths(x_pos, y_pos-1, '-' + curr_strX, Y[y_pos-1] + curr_strY, alignments, X, Y, matrix_trace)
    
    elif(matrix_trace[x_pos][y_pos] == 2): #came from top-left neighbor
        recursive_paths(x_pos-1, y_pos-1, X[x_pos-1] + curr_strX, Y[y_pos-1] + curr_strY, alignments, X, Y, matrix_trace)
    
    elif(matrix_trace[x_pos][y_pos] == 3): #came from top neighbor
        recursive_paths(x_pos-1, y_pos, X[x_pos-1] + curr_strX, '-' + curr_strY, alignments, X, Y, matrix_trace)
    
    elif(matrix_trace[x_pos][y_pos] == 4): #came from left and top-left neighbors
        recursive_paths(x_pos, y_pos-1, '-' + curr_strX, Y[y_pos-1] + curr_strY, alignments, X, Y, matrix_trace)
        recursive_paths(x_pos-1, y_pos-1, X[x_pos-1] + curr_strX, Y[y_pos-1] + curr_strY, alignments, X, Y, matrix_trace)
    
    elif(matrix_trace[x_pos][y_pos] == 5): #came from top-left and top neighbors
        recursive_paths(x_pos-1, y_pos-1, X[x_pos-1] + curr_strX, Y[y_pos-1] + curr_strY, alignments, X, Y, matrix_trace)
        recursive_paths(x_pos-1, y_pos, X[x_pos-1] + curr_strX, '-' + curr_strY, alignments, X, Y, matrix_trace)
    
    elif(matrix_trace[x_pos][y_pos] == 6): #came from left and top neighbors
        recursive_paths(x_pos, y_pos-1, '-' + curr_strX, Y[y_pos-1] + curr_strY, alignments, X, Y, matrix_trace)
        recursive_paths(x_pos-1, y_pos, X[x_pos-1] + curr_strX, '-' + curr_strY, alignments, X, Y, matrix_trace)
    
    else: #came from all three neighbors
        recursive_paths(x_pos, y_pos-1, '-' + curr_strX, Y[y_pos-1] + curr_strY, alignments, X, Y, matrix_trace)
        recursive_paths(x_pos-1, y_pos-1, X[x_pos-1] + curr_strX, Y[y_pos-1] + curr_strY, alignments, X, Y, matrix_trace)
        recursive_paths(x_pos-1, y_pos, X[x_pos-1] + curr_strX, '-' + curr_strY, alignments, X, Y, matrix_trace)

## Main Function

In [4]:
#takes input of sequences X and Y, nucleotide match score, nucleotide mismatch penalty, and gap penalty
#outputs scoring matrix, trace-back matrix, and all optimal alignments

def pairwise_alignment(X: list, Y: list, s_match: int, p_mismatch: int, p_gap: int):
    
    #filling the scoring matrix and trace-back matrix
    matrix, matrix_trace = fill_matrices(X, Y, s_match, p_mismatch, p_gap)
    
    #recursively back-tracing through optimal paths and storing all optimal alignments
    alignments = []
    recursive_paths(len(X), len(Y), '', '', alignments, X, Y, matrix_trace)
    
    #return scoring matrix, trace-back matrix, and optimal alignment(s)
    return matrix, matrix_trace, alignments

In [5]:
#function to display alignment results
#input takes score matrix, trace-back matrix, and list of optimal alignments
def alignment_display(scr_matrix: np.ndarray, tb_matrix: np.ndarray, opt_alignments: list) -> None:
    print("Scoring Matrix:", scr_matrix, sep='\n')
    print()
    print("Traceback Matrix:", tb_matrix, sep='\n')
    print()
    print("All Optimal Alignments:")
    for align in opt_alignments:
        print(align[1])
        print(align[0],"\n")

**Short Example:** <br>
Sequence X: ACGA <br>
Sequence Y: CACT <br>
Nucleotide Match Score: 3 <br>
Nucleotide Mismatch Penalty: -1 <br>
Gap Penalty: -2 <br>


In [6]:
alignment_display(*pairwise_alignment('ACGA', 'CACT', 3, -1, -2))

Scoring Matrix:
[[ 0 -2 -4 -6 -8]
 [-2 -1  1 -1 -3]
 [-4  1 -1  4  2]
 [-6 -1  0  2  3]
 [-8 -3  2  0  1]]

Traceback Matrix:
[[1 1 1 1 1]
 [3 2 2 1 1]
 [3 2 6 2 1]
 [3 3 2 3 2]
 [3 3 2 6 5]]

All Optimal Alignments:
-ACGA
CAC-T 

-ACGA
CACT- 



**Longer Example:** &nbsp; &nbsp; *With Sequences of Different Lengths* <br>
Sequence X: CCGAGCGGTGCAG <br>
Sequence Y: CAAGGGTTCTGGTCCGC <br>
Nucleotide Match Score: 2 <br>
Nucleotide Mismatch Penalty: -1 <br>
Gap Penalty: -1 <br>

In [7]:
alignment_display(*pairwise_alignment('CCGAGCGGTGCAG', 'CAAGGGTTCTGGTCCGC', 2, -1, -1))

Scoring Matrix:
[[  0  -1  -2  -3  -4  -5  -6  -7  -8  -9 -10 -11 -12 -13 -14 -15 -16 -17]
 [ -1   2   1   0  -1  -2  -3  -4  -5  -6  -7  -8  -9 -10 -11 -12 -13 -14]
 [ -2   1   1   0  -1  -2  -3  -4  -5  -3  -4  -5  -6  -7  -8  -9 -10 -11]
 [ -3   0   0   0   2   1   0  -1  -2  -3  -4  -2  -3  -4  -5  -6  -7  -8]
 [ -4  -1   2   2   1   1   0  -1  -2  -3  -4  -3  -3  -4  -5  -6  -7  -8]
 [ -5  -2   1   1   4   3   3   2   1   0  -1  -2  -1  -2  -3  -4  -4  -5]
 [ -6  -3   0   0   3   3   2   2   1   3   2   1   0  -1   0  -1  -2  -2]
 [ -7  -4  -1  -1   2   5   5   4   3   2   2   4   3   2   1   0   1   0]
 [ -8  -5  -2  -2   1   4   7   6   5   4   3   4   6   5   4   3   2   1]
 [ -9  -6  -3  -3   0   3   6   9   8   7   6   5   5   8   7   6   5   4]
 [-10  -7  -4  -4  -1   2   5   8   8   7   6   8   7   7   7   6   8   7]
 [-11  -8  -5  -5  -2   1   4   7   7  10   9   8   7   6   9   9   8  10]
 [-12  -9  -6  -3  -3   0   3   6   6   9   9   8   7   6   8   8   8   9]
 [-13 -10

This code can be applied to other sequences to find optimal global alignment.